# Energy Reconstruction Using CNN

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import math

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Attention

from tensorflow.keras.callbacks import ModelCheckpoint

from data_tools import load_preprocessed, dataPrep, nameModel

## FIX THIS!
simPrefix = 'D:\icecube\sim_data'

In [2]:
# Name for model
key = 'Conv_Product'

# Data preparation: no merging of charge (q), no time layers included (t=False), data normalized from 0-1
prep = {'q':'product', 't':False, 'normed':False, }#cosz = True}

In [3]:
x, y = load_preprocessed(simPrefix, 'train')
print(y.keys())

Percentage of events with a NaN: 2.68
dict_keys(['comp', 'energy', 'dir', 'plane_dir', 'laputop_dir', 'small_dir'])


In [47]:
import keras_tuner as kt
def model_builder(hp):

    
o = layers.Conv2D(64, kernel_size=3, activation='relu', input_shape=(10,10,1))(inputs)
#o = layers.LayerNormalization(axis=1)(o)
o = layers.Conv2D(32, kernel_size=3, activation='relu')(o)
#o = layers.LayerNormalization(axis=1)(o)
o = layers.Conv2D(16, kernel_size=3, activation='relu')(o)
#o = layers.LayerNormalization(axis=1)(o)
o = Flatten()(o)
outputs = Dense(2)(o)
#outputs = layers.GlobalAveragePooling2D()(o)





model = Model(inputs=inputs, outputs=outputs, name="attn_test")
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

IndentationError: expected an indented block (Temp/ipykernel_1480/1366296848.py, line 5)

In [10]:
from tensorflow.keras import Input, layers, Model
import tensorflow as tf

inputs = Input(shape=(10,10,1))

o = layers.Conv2D(64, kernel_size=3, activation='relu', input_shape=(10,10,1))(inputs)
o = layers.BatchNormalization(axis=1)(o)
o = layers.Conv2D(32, kernel_size=3, activation='relu')(o)
o = layers.BatchNormalization(axis=1)(o)
o = layers.Conv2D(16, kernel_size=3, activation='relu')(o)
o = layers.BatchNormalization(axis=1)(o)
o = Flatten()(o)
outputs = Dense(2)(o)
#outputs = layers.GlobalAveragePooling2D()(o)





model = Model(inputs=inputs, outputs=outputs, name="attn_test")


model.summary()

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])




Model: "attn_test"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 10, 10, 1)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 64)          640       
_________________________________________________________________
batch_normalization_3 (Batch (None, 8, 8, 64)          32        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 32)          18464     
_________________________________________________________________
batch_normalization_4 (Batch (None, 6, 6, 32)          24        
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 4, 16)          4624      
_________________________________________________________________
batch_normalization_5 (Batch (None, 4, 4, 16)          16

In [8]:
energy = y['energy']
comp = y['comp']
theta, phi = y['dir'].transpose()
nevents = len(energy)
trainCut = (np.random.uniform(size=nevents) < 0.85)
testCut = np.logical_not(trainCut)

# Establish arrays to be trained on
x_i = dataPrep(x, y, **prep)



temp_y = energy


In [13]:
history = model.fit(x_i[trainCut], temp_y[trainCut], validation_data=(x_i[testCut], temp_y[testCut]), epochs=3, batch_size = 1000)



Epoch 1/3
468/468 [==============================] - 96s 205ms/step - loss: 0.0395 - mse: 0.0395 - val_loss: 0.0395 - val_mse: 0.0395
Epoch 2/3
468/468 [==============================] - 94s 200ms/step - loss: 0.0389 - mse: 0.0389 - val_loss: 0.0388 - val_mse: 0.0388
Epoch 3/3
468/468 [==============================] - 96s 204ms/step - loss: 0.0386 - mse: 0.0386 - val_loss: 0.0394 - val_mse: 0.0394


In [ ]:
# Save model to file for easy loading
## WHERE ARE YOU SAVING TO?
model.save('model_%s.h5' % key)

In [ ]:
history.history.get_weights()

In [46]:
for i in range(9):
    #print(model.get_layer(index = i).get_weights())
    print(i)


0
1
2
3
4
5
6
7
8


In [6]:
print(y)



{'comp': array([ 1,  1,  1, ..., 56, 56, 56], dtype=int64), 'energy': array([5.13847214, 5.13847214, 5.34288138, ..., 6.91190239, 6.91190239,
       6.91190239]), 'dir': array([[2.92503931, 3.90685368],
       [2.92503931, 3.90685368],
       [2.7859734 , 4.8815546 ],
       ...,
       [2.17614535, 3.62439525],
       [2.17614535, 3.62439525],
       [2.17614535, 3.62439525]]), 'plane_dir': array([[2.888150384237141, 4.041238923755302],
       [2.9104651693856254, 3.8575433691584395],
       [2.7652575606033416, 4.843763269685228],
       ...,
       [2.1967368639002016, 3.5506960777999406],
       [2.2040490115868696, 3.5975026573490676],
       [2.1792771408108615, 3.6343438039541875]], dtype=object), 'laputop_dir': array([[None, None],
       [None, None],
       [None, None],
       ...,
       [2.169848069946557, 3.6150110129485924],
       [None, None],
       [2.193166465600582, 3.6230477763857687]], dtype=object), 'small_dir': array([[2.888150384237141, 4.041238923755302],
   